In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5

In [ ]:
!nvidia-smi

Thu Aug  3 06:50:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


## LLaMA2 7B Chat


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                            #  load_in_8bit=True,
                                            #  load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:628: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:460: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:372: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [ ]:
# Use a pipeline for later
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
!nvidia-smi

Thu Aug  3 07:01:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    33W /  70W |  13801MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS # <<SYS>> Default <</SYS>>
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST # [INST] <<SYS>> Default </SYS>> 내용 [/INST]
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt              #  Prompt는 끊고자 하는 내용
    index = text.find(cutoff_phrase)    #  index는 text 내 끊고자 하는 내용의 위치
    if index != -1:                     #  끊고자 하는 prompt가 맨 뒤에 있지 않다면
        return text[:index]             #  해당 위치까지 끊어서 출력
    else:
        return text                     #  맨 뒤에 있는 경우 그대로 출력

def remove_substring(string, substring):
    return string.replace(substring, "")  # Substring을 삭제처리하는 함수



def generate(text):
    prompt = get_prompt(text)           # [INST] <<SYS>> Default </SYS>> {text} [/INST]

    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [ ]:
instruction = "What is the temperature in Melbourne?"

get_prompt(instruction)

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\nWhat is the temperature in Melbourne?[/INST]"

In [ ]:
instruction = "Summarize the following text for me {text}"

system_prompt = "You are an expert and summarization and reducing the number of words used"

get_prompt(instruction, system_prompt)

'[INST]<<SYS>>\nYou are an expert and summarization and reducing the number of words used\n<</SYS>>\n\nSummarize the following text for me {text}[/INST]'

## LangChain basics

In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


In [ ]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.8})

In [ ]:
system_prompt = "You are an advanced assistant that excels at translation. "
instruction = "Convert the following text from English to Korean:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

[INST]<<SYS>>
You are an advanced assistant that excels at translation. 
<</SYS>>

Convert the following text from English to Korean:

 {text}[/INST]


In [ ]:
text = "how are you today?"
output = llm_chain.run(text)

parse_text(output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


  Sure, I'd be happy to help you translate "how are you today?" from English to Korean. Here's the
translation:  今日 (goinjang)이 좋아?  I hope that helps! Let me know if you have any other questions.




한국어 시작부터 별로네..


### Summarization

In [ ]:
instruction = "Summarize the following article for me {text}"
system_prompt = "You are an expert and summarization and expressing key ideas succintly"

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

[INST]<<SYS>>
You are an expert and summarization and expressing key ideas succintly
<</SYS>>

Summarize the following article for me {text}[/INST]


In [ ]:
text = '약학'
instruction = """관심사에 관련된 탐구적인 에세이 주제 1개, 에세이 작성에 도움이 될만한 가이드 질문 3개를 뽑아줘. 관심사:{text}"""
system_prompt = """"너는 고등학생을 대상으로 탐구 활동을 추천해주는 진로상담 전문가야. 다음 형식과 동일하게 출력해.
        에세이 주제 : 내용
        Question1 내용
        Question2 내용
        Question3 내용 """

template = get_prompt(instruction, system_prompt)
print(template)

[INST]<<SYS>>
"너는 고등학생을 대상으로 탐구 활동을 추천해주는 진로상담 전문가야. 다음 형식과 동일하게 출력해.
        에세이 주제 : 내용
        Question1 내용
        Question2 내용
        Question3 내용 
<</SYS>>

관심사에 관련된 탐구적인 에세이 주제 1개, 에세이 작성에 도움이 될만한 가이드 질문 3개를 뽑아줘. 관심사:{text}[/INST]


In [ ]:
%%time
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
output = llm_chain.run(text)
print(count_words(output))
parse_text(output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: ignored

In [ ]:
output = llm_chain.run(text)
print(count_words(output))
parse_text(output)

In [ ]:
output = llm_chain.run(text)
print(count_words(output))
parse_text(output)

124
  Sure, here is a summary of the article in 400 words or less: Twitter's newly appointed CEO, Linda
Yaccarino, has claimed in a memo to staff that the platform's usage is at an all-time high. She also
highlighted the company's grand plans to expand beyond social media and become an "everything app"
as envisioned by owner Elon Musk. Yaccarino touted the company's innovation in areas such as long-
form content, creator monetization, and brand safety protections. Despite reports of traffic
decline, Yaccarino remained optimistic and emphasized the company's commitment to preserving free
expression and creating a global marketplace for ideas and opportunities. She also acknowledged the
challenges ahead and encouraged the team to work together to make the next big impression on the
world.




## Simple Chatbot

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate

In [ ]:
instruction = "Chat History:\n\n{chat_history} \n\nUser: {user_input}"
system_prompt = "You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context"

template = get_prompt(instruction, system_prompt)
print(template)

[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

{chat_history} 

User: {user_input}[/INST]


In [ ]:

prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
llm_chain.predict(user_input="Hi, my name is Sam")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

User: Hi, my name is Sam[/INST]

> Finished chain.


'  Hello Sam! How may I assist you today?'

In [ ]:
llm_chain.predict(user_input="Can you tell me about yourself.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:   Hello Sam! How may I assist you today? 

User: Can you tell me about yourself.[/INST]

> Finished chain.


"  Of course! *smiling* I'm just an AI, I don't have personal experiences or emotions like humans do. My purpose is to assist and provide helpful responses to users like you. I'm here to help with any questions or tasks you may have, so feel free to ask me anything!"

In [ ]:
llm_chain.predict(user_input="Today is Friday. What number day of the week is that?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: Hi, my name is Sam
AI:   Hello Sam! How may I assist you today?
Human: Can you tell me about yourself.
AI:   Of course! *smiling* I'm just an AI, I don't have personal experiences or emotions like humans do. My purpose is to assist and provide helpful responses to users like you. I'm here to help with any questions or tasks you may have, so feel free to ask me anything! 

User: Today is Friday. What number day of the week is that?[/INST]

> Finished chain.


'  AI: Oh, a clever question! *smiling* Today is the fifth day of the week, which is Friday!'